In [1]:
from random import randint
import threading
import time

class Cafe:
    def __init__(self, num_tables, num_waiters):
        self.tables = [Table() for _ in range(num_tables)]
        self.waiters = [Waiter() for _ in range(num_waiters)]
        self.customers = []
        self.lock = threading.Lock()

    def simulate_day(self, opening_time, closing_time):
        current_time = opening_time

        waiter_threads = [threading.Thread(target=self.waiter_work, args=(waiter,)) for waiter in self.waiters]
        for thread in waiter_threads:
            thread.start()

        while current_time <= closing_time:
            with self.lock:
                if randint(0, 10) > 5:
                    num_customers = randint(1, 4)
                    customers = [Customer(self.tables) for _ in range(num_customers)]
                    self.customers.extend(customers)

                self.customers = [customer for customer in self.customers if not customer.leave_cafe()]

                print(f"Time: {current_time}, Occupied Tables: {len([table for table in self.tables if table.is_occupied()])}")

            current_time += 1
            time.sleep(1)

        for thread in waiter_threads:
            thread.join()

    def waiter_work(self, waiter):
        while True:
            with self.lock:
                waiter.serve_customers(self.customers)

class Table:
    def __init__(self):
        self.occupied = False

    def occupy(self):
        self.occupied = True

    def vacate(self):
        self.occupied = False

    def is_occupied(self):
        return self.occupied

class Waiter:
    def __init__(self):
        pass

    def serve_customers(self, customers):
        for customer in customers:
            if not customer.is_served() and not customer.is_waiting():
                customer.serve()

class Customer:
    def __init__(self, tables):
        self.tables = tables
        self.table = None
        self.served = False
        self.waiting = False
        self.patience = randint(5, 20)

    def leave_cafe(self):
        if self.is_waiting() and self.patience == 0:
            print("Customer left due to impatience.")
            return True
        return False

    def is_served(self):
        return self.served

    def is_waiting(self):
        return self.waiting

    def serve(self):
        available_tables = [table for table in self.tables if not table.is_occupied()]
        if available_tables:
            selected_table = available_tables[randint(0, len(available_tables) - 1)]
            selected_table.occupy()
            self.table = selected_table
            self.served = True
            print("Customer seated.")
        else:
            self.waiting = True
            self.patience -= 1
            print("Customer waiting.")


if __name__ == "__main__":
    cafe = Cafe(num_tables=10, num_waiters=2)
    cafe.simulate_day(opening_time=10, closing_time=22)



Time: 10, Occupied Tables: 0
Time: 11, Occupied Tables: 0
Customer seated.
Customer seated.
Customer seated.
Time: 12, Occupied Tables: 3
Time: 13, Occupied Tables: 3
Time: 14, Occupied Tables: 3
Customer seated.
Customer seated.
Customer seated.
Time: 15, Occupied Tables: 6
Customer seated.
Customer seated.
Customer seated.
Time: 16, Occupied Tables: 9
Customer seated.
Customer waiting.
Customer waiting.
Customer waiting.
Time: 17, Occupied Tables: 10
Customer waiting.
Customer waiting.



KeyboardInterrupt

